In [1]:
%pylab inline
from astropy.table import Table, hstack,vstack, Column, QTable
from astropy.io import fits
import astropy.units as u
from astropy.coordinates import SkyCoord
from astropy.coordinates import FK5
from astropy.time import Time
from astropy.coordinates import Angle
from astropy.io import ascii
import numpy.ma as ma

Populating the interactive namespace from numpy and matplotlib


In [2]:
tsc = Table.read('/Users/JanviMadhani/thaw_star_catalog/finaltable.csv')

In [3]:
### Add in 1950 coordinates columns
RA1950 = Column(np.arange(len(tsc)), name='RA1950')
tsc.add_column(RA1950, index=9)
tsc['RA1950'] = tsc['RA1950'].astype(str)

Dec1950 = Column(np.arange(len(tsc)), name='Dec1950')
tsc.add_column(Dec1950, index=10)
tsc['Dec1950'] = tsc['Dec1950'].astype(str)

## Precess to 1950 Coordinates using YPC 1900 Coordinates

In [4]:
def convert_to_J1950(ra,dec,start_equinox='J1900'):
    """
    Give function RA hour, min, seconds,
                  Dec deg, min, seconds
                  Starting equinox: default = 1900
    Get RA in format: 00h00m00.00s
    Dec in format: 00d00m00.00s
    """
        
    if ra:
        
        fk0c = SkyCoord(ra, dec, unit=(u.hourangle, u.deg), frame=FK5(equinox=Time(start_equinox)))
        fk0_1950 = FK5(equinox='J1950')
        c = fk0c.transform_to(fk0_1950,type)
        string = c.to_string('hmsdms',sep=(' ',' '))
        ra1950 = string[:12]
        dec1950 = string[14:]
        
        
    else:
        ra1950,dec1950 = 0,0
        c = 0 

    
    return(ra1950,dec1950)

In [5]:
ra = '00 13 0.00'
dec = '+36 14 0.00'
convert_to_J1950(ra,dec)

('00 15 36.740', '+36 30 40.4597')

In [6]:
for i in range(len(tsc)):
    ra = tsc['YPC RA1900'][i]
    dec = tsc['YPC Dec1900'][i]
    ra1950,dec1950 = convert_to_J1950(ra,dec)
    tsc['RA1950'][i] = ra1950
    tsc['Dec1950'][i] = dec1950

In [7]:
tsc.show_in_notebook()

idx,Thaw Number,YPC Number,Thaw Name,YPC Name,BD Number,Thaw RA1900,Thaw Dec1900,YPC RA1900,YPC Dec1900,RA1950,Dec1950,RA2000,Dec2000,Thaw B,YPC (B-V),YPC V,Thaw Spectrum,YPC Spectrum,Thaw π Rel.,Thaw π Rel. Error,Δπ,Thaw π Abs.,Yale π Abs.,Yale π Abs. Error,Yale Tot. PM,No. of Thaw Observations,Thaw Footnotes
0,1,51.00,Sigma Andromedae,SIG And,+35°44,00 13 0.00,+36 14 0.00,00 13 06.0,+36 13 51,00 15 42.761,+36 30 31.4322,00 18 19.7,+36 47 07,4.5,4.52,0.05,A2,A2 V,12,8,--,--,19,12.5,0.074,--,--
1,2,158.00,Delta Piscium,DEL Psc,+6°107,00 43 0.00,+7 02 0.00,00 43 29.5,+07 02 27,00 46 04.803,+07 18 50.3317,00 48 40.9,+07 35 06,4.5,4.43,1.5,K5,K4 IIIb,12,7,--,--,17,9.4,0.095,--,--
2,3,209.00,Epsilon Piscium,EPS Psc,+7°153,00 58 0.00,+7 21 0.00,00 57 45.0,+07 21 06,01 00 20.907,+07 37 15.3213,01 02 56.5,+07 53 24,4.5,4.28,0.96,G5,K0 III,23,8,--,--,26,6.5,0.085,--,--
3,4,219.00A,Mu Cassiopeiae,MU Cas,+54°223,01 02 0.00,+54 26 0.00,01 01 36.7,+54 25 47,01 04 35.902,+54 41 51.6498,01 08 16.2,+54 55 11,5.3,5.17,0.69,G5,G5 Vb,143,9,--,--,133,1.7,3.77,--,--
4,5,368.00,Omicron Piscium,OMI Psc,+8°273,01 40 0.00,+8 39 0.00,01 40 06.6,+08 39 16,01 42 44.689,+08 54 21.8334,01 45 23.6,+09 09 28,4.5,4.26,0.96,G5,G8 III,12,6,--,--,21,7.7,0.089,--,--
5,6,394.00,Beta Arietis,BET Ari,+20°306,01 49 0.00,+20 19 0.00,01 49 06.8,+20 19 09,01 51 52.018,+20 33 57.1538,01 54 38.4,+20 48 28,2.7,2.64,0.13,A5,A5 V,72,8,--,--,67,7.8,0.144,--,--
6,7,567.00,39 Arietis,39 Ari,+28°462,02 42 0.00,+28 50 0.00,02 41 57.0,+28 49 55,02 44 54.840,+29 02 33.1509,02 47 54.5,+29 14 49,4.6,4.51,1.11,K,K1.5 III,22,6,--,--,20,1.9,0.207,--,--
7,8,897.00A,37 Tauri,37 Tau,+21°585,03 59 0.00,+21 49 0.00,03 58 46.8,+21 48 31,04 01 43.724,+21 56 51.2328,04 04 41.7,+22 04 54,4.5,4.36,1.07,K,K0+ III-*,-22,9,--,--,12,5.7,0.107,--,--
8,9,911.00,50 Persei,50 Per,+37°882,04 02 0.00,+37 47 0.00,04 01 56.5,+37 46 41,04 05 15.538,+37 54 48.4982,04 08 36.6,+38 02 23,5.6,5.51,0.46,F,F7 V,43,8,--,--,47,12.5,0.261,--,--
9,10,1475.00,Mu Geminorum,* mu. Gem,+22°1304,06 17 0.00,+22 34 0.00,06 16 54.5,+22 33 54,06 19 55.834,+22 32 33.5102,06 22 57.5,+22 30 49,3.2,2.88,1.64,Ma,M3 IIIab,30,8,--,--,18,5.3,0.125,--,--


In [8]:
tsc["Yale " + u"\u03c0" + " Abs."] = tsc["Yale " + u"\u03c0" + " Abs."].astype(str)
tsc["Yale " + u"\u03c0" + " Abs. Error"] = tsc["Yale " + u"\u03c0" + " Abs. Error"].astype(str)
tsc['YPC Spectrum'] = tsc['YPC Spectrum'].astype(str)
tsc['YPC V'] = tsc['YPC V'].astype(str)
tsc['YPC (B-V)'] = tsc['YPC (B-V)'].astype(str)

for i in range(len(tsc)):
    s1 = tsc['YPC Spectrum'][i]
    s1 = (str(s1.astype(str)))
    tsc['YPC Spectrum'][i] = s1

    s2 = tsc['YPC V'][i]
    s2 = (str(s2.astype(str)))    
    tsc['YPC V'][i] = s2 
    
    s3 = tsc['YPC (B-V)'][i]
    s3 = (str(s3.astype(str)))  
    tsc['YPC (B-V)'][i] = s3

    

In [9]:
type(tsc[1]['YPC V'])

numpy.str_

In [10]:
tsc.show_in_notebook()

idx,Thaw Number,YPC Number,Thaw Name,YPC Name,BD Number,Thaw RA1900,Thaw Dec1900,YPC RA1900,YPC Dec1900,RA1950,Dec1950,RA2000,Dec2000,Thaw B,YPC (B-V),YPC V,Thaw Spectrum,YPC Spectrum,Thaw π Rel.,Thaw π Rel. Error,Δπ,Thaw π Abs.,Yale π Abs.,Yale π Abs. Error,Yale Tot. PM,No. of Thaw Observations,Thaw Footnotes
0,1,51.00,Sigma Andromedae,SIG And,+35°44,00 13 0.00,+36 14 0.00,00 13 06.0,+36 13 51,00 15 42.761,+36 30 31.4322,00 18 19.7,+36 47 07,4.5,4.52,0.05,A2,A2 V,12,8,--,--,19,12.5,0.074,--,--
1,2,158.00,Delta Piscium,DEL Psc,+6°107,00 43 0.00,+7 02 0.00,00 43 29.5,+07 02 27,00 46 04.803,+07 18 50.3317,00 48 40.9,+07 35 06,4.5,4.43,1.5,K5,K4 IIIb,12,7,--,--,17,9.4,0.095,--,--
2,3,209.00,Epsilon Piscium,EPS Psc,+7°153,00 58 0.00,+7 21 0.00,00 57 45.0,+07 21 06,01 00 20.907,+07 37 15.3213,01 02 56.5,+07 53 24,4.5,4.28,0.96,G5,K0 III,23,8,--,--,26,6.5,0.085,--,--
3,4,219.00A,Mu Cassiopeiae,MU Cas,+54°223,01 02 0.00,+54 26 0.00,01 01 36.7,+54 25 47,01 04 35.902,+54 41 51.6498,01 08 16.2,+54 55 11,5.3,5.17,0.69,G5,G5 Vb,143,9,--,--,133,1.7,3.77,--,--
4,5,368.00,Omicron Piscium,OMI Psc,+8°273,01 40 0.00,+8 39 0.00,01 40 06.6,+08 39 16,01 42 44.689,+08 54 21.8334,01 45 23.6,+09 09 28,4.5,4.26,0.96,G5,G8 III,12,6,--,--,21,7.7,0.089,--,--
5,6,394.00,Beta Arietis,BET Ari,+20°306,01 49 0.00,+20 19 0.00,01 49 06.8,+20 19 09,01 51 52.018,+20 33 57.1538,01 54 38.4,+20 48 28,2.7,2.64,0.13,A5,A5 V,72,8,--,--,67,7.8,0.144,--,--
6,7,567.00,39 Arietis,39 Ari,+28°462,02 42 0.00,+28 50 0.00,02 41 57.0,+28 49 55,02 44 54.840,+29 02 33.1509,02 47 54.5,+29 14 49,4.6,4.51,1.11,K,K1.5 III,22,6,--,--,20,1.9,0.207,--,--
7,8,897.00A,37 Tauri,37 Tau,+21°585,03 59 0.00,+21 49 0.00,03 58 46.8,+21 48 31,04 01 43.724,+21 56 51.2328,04 04 41.7,+22 04 54,4.5,4.36,1.07,K,K0+ III-*,-22,9,--,--,12,5.7,0.107,--,--
8,9,911.00,50 Persei,50 Per,+37°882,04 02 0.00,+37 47 0.00,04 01 56.5,+37 46 41,04 05 15.538,+37 54 48.4982,04 08 36.6,+38 02 23,5.6,5.51,0.46,F,F7 V,43,8,--,--,47,12.5,0.261,--,--
9,10,1475.00,Mu Geminorum,* mu. Gem,+22°1304,06 17 0.00,+22 34 0.00,06 16 54.5,+22 33 54,06 19 55.834,+22 32 33.5102,06 22 57.5,+22 30 49,3.2,2.88,1.64,Ma,M3 IIIab,30,8,--,--,18,5.3,0.125,--,--


## Create Subtables

In [29]:
#define colnames

colnames = ['TPC No', 'AO Result Published', 'AO Rel. Parallax (err)', 'AO Abs. Parallax (err)', 'YPC No', 'YPC Abs. Parallax (err)', 
           'AO Name', 'YPC Name', 'BD Name', 'RA2000','Dec2000','AO RA1900','AO Dec1900','YPC Total Proper Motion','YPC Spectrum',
           'YPC V Mag', 'YPC B-V Mag','AO Obs No, Quality, Date']
vals = [i for i in range(len(colnames))]
for i in range(1):
    if tsc[i]['Thaw RA1900']:
        subtable = Table(names=colnames,dtype = [object,object,object,object,object,object,object,object,object,
                                                object,object,object,object,object,object,object,object,object])
        tpc_no = tsc[i]['Thaw Number']
        
        ao_pub_date = 'yyyymmdd'
        ao_rel_p = tsc[i]["Thaw " + u"\u03c0" + " Rel."] + str(' (') + tsc[i]['Thaw ' + u"\u03c0" + ' Rel. Error'] + str(')')
        ao_abs_p = ''
        ypc_no = tsc[i]['YPC Number']
        ypc_abs_p = tsc[i]["Yale " + u"\u03c0" + " Abs."] + str(" (") + tsc[i]["Yale " + u"\u03c0" + " Abs. Error"] + str(")")
        ao_name = tsc[i]['Thaw Name']
        ypc_name = tsc[i]['YPC Name']
        bd = tsc[i]['BD Number']
        ra2000 = tsc[i]['RA2000']
        dec2000 = tsc[i]['Dec2000']
        ra1900 = tsc[i]['Thaw RA1900']
        dec1900 = tsc[i]['Thaw Dec1900']
        pm = tsc[i]['Yale Tot. PM']
        
        
        subtable['YPC Spectrum'] = []
        subtable['YPC Spectrum'] = subtable['YPC Spectrum'].astype(object)
        ypc_spec = tsc['YPC Spectrum'][i]
        
        
  
        subtable['YPC V Mag'] = []
        subtable['YPC V Mag'] = subtable['YPC V Mag'].astype(object)
        ypc_v = tsc['YPC V'][i]
        
        
        subtable['YPC B-V Mag'] = []
        subtable['YPC B-V Mag'] = subtable['YPC B-V Mag'].astype(object)
        ypc_bv = tsc['YPC (B-V)'][i]        
        
        observations = []
        observations.append('xxxxxx q yyyymmdd')
        
        print(subtable.colnames)
        
        subtable.add_row((tpc_no,ao_pub_date,ao_rel_p,ao_abs_p,ypc_no,ypc_abs_p,ao_name,ypc_name,bd,
                         ra2000,dec2000,ra1900,dec1900,pm,ypc_spec,ypc_v,ypc_bv,observations))
        
        subtable.show_in_browser()
        
    
    

['TPC No', 'AO Result Published', 'AO Rel. Parallax (err)', 'AO Abs. Parallax (err)', 'YPC No', 'YPC Abs. Parallax (err)', 'AO Name', 'YPC Name', 'BD Name', 'RA2000', 'Dec2000', 'AO RA1900', 'AO Dec1900', 'YPC Total Proper Motion', 'YPC Spectrum', 'YPC V Mag', 'YPC B-V Mag', 'AO Obs No, Quality, Date']
